## Импорт библиотек

In [1]:
import datetime
from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np

## Загрузка данных

In [2]:
df_raw= pd.read_csv(r"C:\Users\Stepan\Desktop\ВУЗ\Клиент-серверные\папка датафреймов\df_after_cleaning.csv")

## Преобразование категориальных признаков.

Преобразование даты. Преобразуем в числовый признак, отражающий колиечство дней, прошедших со дня публикации.

In [3]:
def datedelta(x):
    y, m, d= [int(i) for i in x.split('-')]
    dat= datetime.date(y, m, d)
    return (datetime.date.today()-dat).days
df_raw['date']= df_raw['date'].transform(lambda x: datedelta(x))

Преобразование координат в расстояние до Кремля

In [4]:
def raddeg(r):
    d= r* 180 /np.pi
    return d

def degrad(d):
    r= d* np.pi / 180
    return r

def coord_distance(w1, l1, w2, l2):
    t= l1- l2
    d= 60* 1.1515* raddeg(np.arccos(
            (np.sin(degrad(w1))* np.sin(degrad(w2))) + 
            (np.cos(degrad(w1))* np.cos(degrad(w2)) * np.cos(degrad(t)))))    

    return round(d* 1.609344, 2)
df_raw['distance_to_Kremlin']=coord_distance(df_raw['coord_width'], df_raw['coord_len'], 55.751462, 37.618790)

Преобразование координат в район с помощью библиотеки geopy

In [5]:
geolocator= Nominatim(user_agent='prohorovstepan17@gmail.com')

district=[]
w, l=list(df_raw['coord_width']), list(df_raw['coord_len'])
for i in range(len(w)):
    coords= [w[i],l[i]] 
    location = geolocator.reverse(coords)
    district.append(location.raw['address'].get('suburb'))
print(len(district))

9354


Преобразование None значений района в ''

In [6]:
for i in range(len(district)):
    if district[i]==None:
        district[i]=''

df_raw['district']=list(map(lambda x: x.replace('район','').strip(),district))
df_raw

,Unnamed: 0,metro_name,time_to_metro,num_rooms,total_area,year_house,kitchen_area,repair,price,floors,num_floor,address,date,coord_width,coord_len,link,distance_to_Kremlin,district
0,1,Ясенево,8.0,0.0,26.0,1700.0,0.0,евро,3174900.0,16.0,16.0,"Москва, ул. Паустовского, 8к3",10,55.599967,37.536365,https://www.avito.ru/moskva/kvartiry/kvartira-...,17.62,Ясенево
1,2,Улица академика Янгеля,31.0,0.0,10.0,1973.0,0.0,требует ремонта,2500000.0,12.0,1.0,"Москва, Медынская ул., 4/1",10,55.592490,37.645717,https://www.avito.ru/moskva/kvartiry/kvartira-...,17.76,Бирюлёво Западное
2,3,Братиславская,25.5,0.0,14.5,1700.0,0.0,косметический,5700000.0,26.0,9.0,"ул. Люблинская, вл. 72, корп. 11",10,55.665770,37.734130,https://www.avito.ru/moskva/kvartiry/kvartira-...,11.96,Люблино
3,4,Панфиловская,8.0,0.0,8.0,1960.0,0.0,дизайнерский,4900000.0,1.0,1.0,"Москва, ул. Маршала Конева, 2",10,55.799952,37.491889,https://www.avito.ru/moskva/kvartiry/kvartira-...,9.59,Щукино
4,6,Рассказовка,31.0,0.0,20.0,2012.0,0.0,косметический,3290000.0,3.0,2.0,"Москва, Новомосковский административный округ,...",10,55.647214,37.327623,https://www.avito.ru/moskva/kvartiry/kvartira-...,21.62,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9349,12098,Спартак,5.0,5.0,137.0,2020.0,18.0,требует ремонта,45000000.0,22.0,16.0,"Москва, Волоколамское ш., 71к5",38,55.815999,37.429959,https://www.avito.ru/moskva/kvartiry/5-k._kvar...,13.82,Покровское-Стрешнево
9350,12099,Кропоткинская,8.0,5.0,230.0,1700.0,25.0,евро,210000000.0,7.0,4.0,"Москва, 2-й Обыденский пер., 3с1",33,55.741546,37.604421,https://www.avito.ru/moskva/kvartiry/5-k._kvar...,1.42,Хамовники
9351,12100,Чкаловская,13.0,5.0,342.0,2023.0,35.0,требует ремонта,318034178.0,9.0,9.0,"Москва, Серебрянический пер., 8",25,55.750019,37.647457,https://www.avito.ru/moskva/kvartiry/5-k._kvar...,1.80,Таганский
9352,12102,Минская,25.5,6.0,583.0,1700.0,58.0,евро,295000000.0,22.0,19.0,"Москва, Минская ул., 1Гк2",33,55.712122,37.499067,https://www.avito.ru/moskva/kvartiry/6-k._kvar...,8.68,Раменки


Количество None значений района

In [8]:
print(len(df_raw[df_raw['district']=='']))

424


Преобразование названия метро в ветку и расстояния до центральных станций

In [9]:
metro_dict={"Арбатско-Покровская": ['Пятницкое шоссе', 'Митино', 'Волоколамская', 'Мякинино','Строгино', 'Крылатское', 'Молодёжная',
'Кунцевская', 'Славянский бульвар',
'Парк Победы', 'Киевская', 'Смоленская', 'Арбатская',
'Площадь революции','Курская','Бауманская',
'Электрозаводская','Семёновская','Партизанская','Измайловская','Первомайская', 'Щёлковская'],
"Бутовская":

[
    'Улица Горчакова',
    'Бульвар адмирала Ушакова',
 'Улица Скобелевская',
 'Улица Старокачаловская',
 'Лесопарковая',
 'Битцевский парк',




],
'Замоскворецкая':

['Ховрино','Беломорская','Речной вокзал','Водный стадион',
 'Войковская','Сокол', 'Аэропорт', 'Динамо', 'Белорусская', 'Маяковская',
 'Тверская', 'Театральная', 'Новокузнецкая', 'Павелецкая', 'Автозаводская',
 'Технопарк', 'Коломенская', 'Каширская', 'Кантемировская', 'Царицыно', 'Орехово',
 'Домодедовская','Красногвардейская','Алма-Атинская'
],
 
'Калининско-Солнцевская':[
    'Новокосино',
    'Новогиреево',
    'Перово',
    'Шоссе Энтузиастов',
    'Авиамоторная',
    'Площадь Ильича',
    'Марксистская',
    'Третьяковская'
],
'Калужско-Рижская':
[
'Новоясеневская',
'Ясенево',
'Тёплый стан',
'Коньково',
'Беляево',
'Калужская',
'Новые Черёмушки',
'Профсоюзная',
'Академическая',
'Ленинский проспект',
'Шаболовская',
'Октябрьская',
'Третьяковская',
'Китай-город',
'Тургеневская',
'Сухаревская',
'Проспект Мира',
'Рижская',
'Алексеевская',
'ВДНХ',
'Ботанический сад',
'Свиблово',
'Бабушкинская',
'Медведково'],
 
'Люблинско-Дмитровская':
[
'Зябликово',
'Шипиловская',
'Борисово',
'Марьино',
'Братиславская',
'Люблино',
'Волжская',
'Печатники',
'Кожуховская',
'Дубровка',
'Крестьянская застава',
'Римская',
'Чкаловская',
'Сретенский бульвар',
'Трубная',
'Достоевская',
'Марьина роща',
'Бутырская', 
'Фонвизинская', 
'Петровско-Разумовская', 
'Окружная', 
'Верхние Лихоборы',
'Селигерская'],
 
'Серпуховско-Тимирязевская':
[
'Бульвар Дмитрия Донского',
'Аннино',
'Улица академика Янгеля',
'Пражская',
'Южная',
'Чертановская',
'Севастопольская',
'Нахимовский Проспект',
'Нагорная',
'Нагатинская',
'Тульская',
'Серпуховская',
'Полянка',
'Боровицкая',
'Чеховская',
'Цветной бульвар',
'Менделеевская',
'Савеловская',
'Дмитровская',
'Тимирязевская',
'Владыкино',
'Отрадное',
'Бибирево',
'Алтуфьево'
],
 
'Солнцевская':
[
'Рассказовка',
'Новопеределкино',
'Боровское шоссе',
'Солнцево',
'Говорово',
'Озёрная',
'Мичуринский проспект',
'Раменки',
'Ломоносовский проспект',
'Минская',
'Парк Победы',
'Деловой центр'],
'Сокольническая':

['Бульвар Рокоссовского',
'Черкизовская',
'Преображенская площадь',
'Сокольники',
'Красносельская',
'Комсомольская',
'Красные ворота',
'Чистые пруды',
'Лубянка',
'Охотный ряд',
'Библиотека им. Ленина',
'Кропоткинская',
'Парк Культуры',
'Фрунзенская',
'Спортивная',
'Воробьёвы горы',
'Университет',
'Проспект Вернадского',
'Юго-Западная',
'Тропарёво',
'Румянцево',
'Саларьево'],
 
'Таганско-Краснопресненская':
[
'Котельники',
'Жулебино',
'Лермонтовский проспект',
'Выхино',
'Рязанский проспект',
'Кузьминки',
'Текстильщики',
'Волгоградский проспект',
'Пролетарская',
'Таганская',
'Китай-город',
'Кузнецкий мост',
'Пушкинская',
'Баррикадная',
'Улица 1905 года',
'Беговая',
'Полежаевская',
'Октябрьское поле',
'Щукинская',
'Спартак',
'Тушинская',
'Сходненская',
'Планерная'],
'Филевская':
[
'Александровский сад',
'Арбатская',
'Смоленская',
'Киевская',
'Студенческая',
'Кутузовская',
'Фили',
'Багратионовская',
'Филевский парк',
'Пионерская',
'Кунцевская',
'Выставочная',
'Международная'],
 
'Кольцевая':
['Парк культуры',
'Октябрьская',
'Калужская ',
'Добрынинская',
'Павелецкая',
'Таганская',
'Курская',
'Комсомольская',
'Проспект Мира',
'Новослободская',
'Краснопресненская',
'Киевская'],
'МЦК':
[
    'Хорошево',
'Новопесчаная',
'Стрешнево',
'Лихоборы',
'Войковская',
'Балтийская',
'Николаевская',
'Окружная',
'Владыкино',
'Ботанический сад',
'Ростокино',
'Белокаменная',
'Бульвар Рокоссовского',
'Локомотив',
'Измайлово',
'Соколиная гора',
'Шоссе энтузиастов',
'Андроновка',
'Угрешская',
'Новохохловская',
'Нижегородская',
'Дубровка',
'Автозаводская',
'ЗИЛ',
'Верхние котлы',
'Крымская',
'Площадь Гагарина',
'Лужники',
'Кутузовская',
'Деловой центр',
'Шелепиха'
],
'Некрасовская':
[
'Электрозаводская',
'Лефортово',
'Авиамоторная',
'Нижегородская',
'Стахановская',
'Окская',
'Юго-Восточная',
'Косино',
'Улица Дмитриевского',
'Некрасовка',
'Лухмановская'
]
      }
dict_metro={i:k for k, v in metro_dict.items() for i in v}
           
           


central_stations={"Арбатско-Покровская":11, "Бутовская":19, "Калужско-Рижская":13,'Замоскворецкая': 10, 
                  'Калининско-Солнцевская': 7, 'Люблинско-Дмитровская':11, 'Серпуховско-Тимирязевская':11,
                  'Солнцевская':11, 'Сокольническая':8, 'Таганско-Краснопресненская':11,'Филевская':0, 'Некрасовская':1
                 }
def metro_line(x):
    try:
        return dict_metro[x.strip()]
    except:
        return 'БКЛ'
def metro_distance(x):
    
    try:
        k=dict_metro[x]
    except:
        k='БКЛ'
    if k=='БКЛ':
        return 6
    if k=='МЦК':
        return 4
    if k=='Кольцевая':
        return 2
    return abs(central_stations[k]-metro_dict[k].index(x))
    
df_raw['metro_line']=list(map(lambda x: metro_line(x), df_raw['metro_name']))
df_raw['metro_distance']=list(map(lambda x: metro_distance(x), df_raw['metro_name']))

Итоговый датафрейм

In [12]:
df_raw=df_raw.drop('Unnamed: 0', axis=1)
df_raw

,metro_name,time_to_metro,num_rooms,total_area,year_house,kitchen_area,repair,price,floors,num_floor,address,date,coord_width,coord_len,link,distance_to_Kremlin,district,metro_line,metro_distance
0,Ясенево,8.0,0.0,26.0,1700.0,0.0,евро,3174900.0,16.0,16.0,"Москва, ул. Паустовского, 8к3",10,55.599967,37.536365,https://www.avito.ru/moskva/kvartiry/kvartira-...,17.62,Ясенево,Калужско-Рижская,12
1,Улица академика Янгеля,31.0,0.0,10.0,1973.0,0.0,требует ремонта,2500000.0,12.0,1.0,"Москва, Медынская ул., 4/1",10,55.592490,37.645717,https://www.avito.ru/moskva/kvartiry/kvartira-...,17.76,Бирюлёво Западное,Серпуховско-Тимирязевская,9
2,Братиславская,25.5,0.0,14.5,1700.0,0.0,косметический,5700000.0,26.0,9.0,"ул. Люблинская, вл. 72, корп. 11",10,55.665770,37.734130,https://www.avito.ru/moskva/kvartiry/kvartira-...,11.96,Люблино,Люблинско-Дмитровская,7
3,Панфиловская,8.0,0.0,8.0,1960.0,0.0,дизайнерский,4900000.0,1.0,1.0,"Москва, ул. Маршала Конева, 2",10,55.799952,37.491889,https://www.avito.ru/moskva/kvartiry/kvartira-...,9.59,Щукино,БКЛ,6
4,Рассказовка,31.0,0.0,20.0,2012.0,0.0,косметический,3290000.0,3.0,2.0,"Москва, Новомосковский административный округ,...",10,55.647214,37.327623,https://www.avito.ru/moskva/kvartiry/kvartira-...,21.62,,Солнцевская,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9349,Спартак,5.0,5.0,137.0,2020.0,18.0,требует ремонта,45000000.0,22.0,16.0,"Москва, Волоколамское ш., 71к5",38,55.815999,37.429959,https://www.avito.ru/moskva/kvartiry/5-k._kvar...,13.82,Покровское-Стрешнево,Таганско-Краснопресненская,8
9350,Кропоткинская,8.0,5.0,230.0,1700.0,25.0,евро,210000000.0,7.0,4.0,"Москва, 2-й Обыденский пер., 3с1",33,55.741546,37.604421,https://www.avito.ru/moskva/kvartiry/5-k._kvar...,1.42,Хамовники,Сокольническая,3
9351,Чкаловская,13.0,5.0,342.0,2023.0,35.0,требует ремонта,318034178.0,9.0,9.0,"Москва, Серебрянический пер., 8",25,55.750019,37.647457,https://www.avito.ru/moskva/kvartiry/5-k._kvar...,1.80,Таганский,Люблинско-Дмитровская,1
9352,Минская,25.5,6.0,583.0,1700.0,58.0,евро,295000000.0,22.0,19.0,"Москва, Минская ул., 1Гк2",33,55.712122,37.499067,https://www.avito.ru/moskva/kvartiry/6-k._kvar...,8.68,Раменки,Солнцевская,2


## Сохранение данных

In [13]:
df_raw.to_csv(r"C:\Users\Stepan\Desktop\ВУЗ\Клиент-серверные\папка датафреймов\df_after_work_with_cat.csv")